In [1]:
import org.locationtech.jts.geom._
import org.apache.spark.sql.types._
import org.locationtech.geomesa.spark.jts._

In [47]:
// import spark.implicits._
// below hack
val spark2: SparkSession = spark
import spark2.implicits._

//,TARGET_FID,suburb_loc,Combustibl,SHAPE_Leng,SHAPE_Area,AU2013Num,IgnProb_bl,RandProb,geometry
/*
Data columns (total 9 columns):
TARGET_FID    1758 non-null int64
suburb_loc    1758 non-null object
Combustibl    1758 non-null int64
SHAPE_Leng    1758 non-null float64
SHAPE_Area    1758 non-null float64
AU2013Num     1758 non-null int64
IgnProb_bl    1758 non-null float64
RandProb      1758 non-null float64
geometry      1758 non-null geometry
*/
val schema = StructType(Array(
  StructField("TARGET_FID",LongType, nullable=false),
  StructField("suburb_location", StringType, nullable=false),
  StructField("Combustible", BooleanType, nullable=false),
  StructField("SHAPE_Length", DoubleType, nullable=false),
  StructField("SHAPE_Area", DoubleType, nullable=false),
  StructField("AU2013Num", LongType, nullable=false),
  StructField("IgnProb", DoubleType, nullable=false),
  StructField("RandProb", DoubleType, nullable=false),
  StructField("polygonText", StringType, nullable=false)))



spark2 = org.apache.spark.sql.SparkSession@f6c545
schema = StructType(StructField(TARGET_FID,LongType,false), StructField(suburb_location,StringType,false), StructField(Combustible,BooleanType,false), StructField(SHAPE_Length,DoubleType,false), StructField(SHAPE_Area,DoubleType,false), StructField(AU2013Num,LongType,false), StructField(IgnProb,DoubleType,false), StructField(RandProb,DoubleType,false), StructField(polygonText,StringType,false))


StructType(StructField(TARGET_FID,LongType,false), StructField(suburb_location,StringType,false), StructField(Combustible,BooleanType,false), StructField(SHAPE_Length,DoubleType,false), StructField(SHAPE_Area,DoubleType,false), StructField(AU2013Num,LongType,false), StructField(IgnProb,DoubleType,false), StructField(RandProb,DoubleType,false), StructField(polygonText,StringType,false))

In [198]:
val filename = "file:///home/jovyan/DEMO/gdf_polygon_1200.csv"
// val dataFile = this.getClass.getClassLoader.getResource(filename).getPath
val df = spark2.read
    .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")
    .option("header", true)
    .option("inferSchema", "true")    
    .csv(filename)
//.schema(schema)
    

filename = file:///home/jovyan/DEMO/gdf_polygon_1200.csv
df = [_c0: int, TARGET_FID: int ... 14 more fields]


[_c0: int, TARGET_FID: int ... 14 more fields]

In [199]:
df.columns

Array(_c0, TARGET_FID, suburb_loc, Combustibl, SHAPE_Leng, SHAPE_Area, AU2013Num, IgnProb_bl, RandProb, geometry, area, centroid, X, Y, d_short, d_long)

In [200]:
//df.show()
//df.count()

spark.withJTS

val alteredDF = df
    .withColumn("polygon", st_polygonFromText($"geometry"))
    .withColumn("centroid", st_makePoint($"X", $"Y"))




alteredDF = [_c0: int, TARGET_FID: int ... 15 more fields]


[_c0: int, TARGET_FID: int ... 15 more fields]

In [201]:
alteredDF.createOrReplaceTempView("chicago")

alteredDF.show()

+---+----------+----------+----------+------------------+------------------+---------+----------------+-----------------+--------------------+------------------+--------------------+------------------+------------------+-------+------+--------------------+
|_c0|TARGET_FID|suburb_loc|Combustibl|        SHAPE_Leng|        SHAPE_Area|AU2013Num|      IgnProb_bl|         RandProb|            geometry|              area|            centroid|                 X|                 Y|d_short|d_long|             polygon|
+---+----------+----------+----------+------------------+------------------+---------+----------------+-----------------+--------------------+------------------+--------------------+------------------+------------------+-------+------+--------------------+
|  0|     58152|    Karori|         1| 91.54329378070867|268.44508528391873|   575000|5.06607709467E-4|0.887780267427702|POLYGON ((1745723...|268.44508528391873|POINT (1745708.76...|1745708.7626952266| 5428056.083294064|    0.0| 

In [202]:
val sqlQuery = "select * from chicago where st_contains(st_makeBBOX(1740000, 5024148, 1780000, 5627600), centroid)"
val resultDataFrame = spark.sql(sqlQuery)

resultDataFrame.count()


sqlQuery = select * from chicago where st_contains(st_makeBBOX(1740000, 5024148, 1780000, 5627600), centroid)
resultDataFrame = [_c0: int, TARGET_FID: int ... 15 more fields]


1758

In [203]:
val target = "st_point(1745712.6242292123, 5428008.229451518)"
val sqlQuery = "select * from chicago where st_distance(centroid, " + target + ") < 50" 
val resultDataFrame = spark.sql(sqlQuery)

resultDataFrame.count()

target = st_point(1745712.6242292123, 5428008.229451518)
sqlQuery = select * from chicago where st_distance(centroid, st_point(1745712.6242292123, 5428008.229451518)) < 50
resultDataFrame = [_c0: int, TARGET_FID: int ... 15 more fields]


13

In [204]:
val centers = alteredDF.select("_c0", "centroid").collect()

centers = Array([0,POINT (1745708.7626952266 5428056.083294064)], [1,POINT (1745652.532815956 5427740.363518546)], [2,POINT (1746195.0530502293 5427985.734522967)], [3,POINT (1745684.194342308 5427602.138345038)], [4,POINT (1745710.3367916616 5428035.235370561)], [5,POINT (1745683.391792311 5427616.884848187)], [6,POINT (1746186.8411587114 5427996.4278080575)], [7,POINT (1745686.4744348666 5427642.380608064)], [8,POINT (1745746.7946155705 5428051.458894832)], [9,POINT (1745660.6090637182 5427659.866465285)], [10,POINT (1746192.041525405 5428031.042856979)], [11,POINT (1745658.1230854467 5427674.307006127)], [12,POINT (1745748.6495584282 5428021.350783869)], [13,POINT (1745651.810717597 5427686.945492816)], [14,POINT (1746176.211299909 5428002.246484985)]...


Array([0,POINT (1745708.7626952266 5428056.083294064)], [1,POINT (1745652.532815956 5427740.363518546)], [2,POINT (1746195.0530502293 5427985.734522967)], [3,POINT (1745684.194342308 5427602.138345038)], [4,POINT (1745710.3367916616 5428035.235370561)], [5,POINT (1745683.391792311 5427616.884848187)], [6,POINT (1746186.8411587114 5427996.4278080575)], [7,POINT (1745686.4744348666 5427642.380608064)], [8,POINT (1745746.7946155705 5428051.458894832)], [9,POINT (1745660.6090637182 5427659.866465285)], [10,POINT (1746192.041525405 5428031.042856979)], [11,POINT (1745658.1230854467 5427674.307006127)], [12,POINT (1745748.6495584282 5428021.350783869)], [13,POINT (1745651.810717597 5427686.945492816)], [14,POINT (1746176.211299909 5428002.246484985)]...

In [126]:
val n = alteredDF.map(x => mymap(x))

n = [value: int]


[value: int]

In [149]:
alteredDF.select("_c0","X","Y").take(2

Array([0,1745708.7626952266,5428056.083294064], [1,1745652.532815956,5427740.363518546])

In [152]:
val adf3 = alteredDF.filter("_c0 < 3")

adf3 = [_c0: int, TARGET_FID: int ... 15 more fields]


[_c0: int, TARGET_FID: int ... 15 more fields]

In [222]:
import org.apache.spark.sql._
// import spark.implicits._ 
def mymap(x: Row): Double = { 
//     println(x)
    return x.getDouble(12)
}

def mymap2(x: Row): Row = { 
//     println(x)
    return Row(x.getInt(0))
}


mymap: (x: org.apache.spark.sql.Row)Double
mymap2: (x: org.apache.spark.sql.Row)org.apache.spark.sql.Row


In [223]:
var res = adf3.map(x => mymap2(x))

Name: Unknown Error
Message: <console>:82: error: Unable to find encoder for type org.apache.spark.sql.Row. An implicit Encoder[org.apache.spark.sql.Row] is needed to store org.apache.spark.sql.Row instances in a Dataset. Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.
       var res = adf3.map(x => mymap2(x))
                         ^

StackTrace: 

In [220]:
res.take(3)

Array(1745708.7626952266, 1745652.532815956, 1746195.0530502293)

In [19]:
val filename = "/home/jovyan/DEMO/gdf_poly_1200.csv"
val dataFile = this.getClass.getClassLoader.getResource(filename).getPath

// val df = spark.read
//    .schema(schema)
//    .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")
//    .csv(dataFile)


//    .option("sep", "-")
// val alteredDF = df
//   .withColumn("polygon", st_polygonFromText($"polygonText"))
  


lastException = null


Name: java.lang.NullPointerException
Message: null
StackTrace: 